In [431]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values, execute_batch
import sys
import importlib
import logging

In [432]:
try: 
    connection=psycopg2.connect(
        host='localhost',
        user='postgres',
        password='123',
        database='healthtech_db',
    )
    print("connection succesfully")
    cursor=connection.cursor()
    cursor.execute("SELECT version()")
    row=cursor.fetchone()
    print(row)
except Exception as ex:
    print(ex)

connection succesfully
('PostgreSQL 18.0 on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)


In [433]:
import sys, importlib

sys.path.append("/Users/youngeddieb/PyCharmProjects/BI-Analytics/BI-Analytics/Pipeline")

import transformation_data
importlib.reload(transformation_data)
from transformation_data import transform_doctors, transform_appointments

# Define paths
project_root = "/Users/youngeddieb/PyCharmProjects/BI-Analytics/BI-Analytics/Pipeline"
doctors_path = f"{project_root}/datasets/Data Enginner's Doctors Excel - VIP Medical Group.xlsx"
appointments_path = f"{project_root}/datasets/Data Engineer's Appointments Excel - VIP Medical Group.xlsx"

# 1) Extract
doctors_df, appointments_df = extract_data(doctors_path, appointments_path)

# 2) Transform (kept only in memory)
doctors_clean = transform_doctors(doctors_df)
appointments_clean = transform_appointments(appointments_df)

# 3) Preview results
display(doctors_clean.head())
display(appointments_clean.head())

2025-11-02 12:49:00,102 | INFO | === START: INGEST ===
2025-11-02 12:49:00,103 | INFO | Reading doctors file from: /Users/youngeddieb/PycharmProjects/BI-Analytics/BI-Analytics/Pipeline/datasets/Data Enginner's Doctors Excel - VIP Medical Group.xlsx
2025-11-02 12:49:00,126 | INFO | Doctors file loaded: 5 rows
2025-11-02 12:49:00,126 | INFO | Reading appointments file from: /Users/youngeddieb/PycharmProjects/BI-Analytics/BI-Analytics/Pipeline/datasets/Data Engineer's Appointments Excel - VIP Medical Group.xlsx
2025-11-02 12:49:00,151 | INFO | Appointments file loaded: 1025 rows
2025-11-02 12:49:00,151 | INFO | === END: INGEST ===
2025-11-02 12:49:00,152 | INFO | === START: INGEST ===
2025-11-02 12:49:00,153 | INFO | Reading doctors file from: /Users/youngeddieb/PyCharmProjects/BI-Analytics/BI-Analytics/Pipeline/datasets/Data Enginner's Doctors Excel - VIP Medical Group.xlsx
2025-11-02 12:49:00,160 | INFO | Doctors file loaded: 5 rows
2025-11-02 12:49:00,161 | INFO | Reading appointments 

/Users/youngeddieb/PycharmProjects/BI-Analytics/BI-Analytics/Pipeline/ingest_data.py
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'appointments_path', 'creating_logger', 'doctors_path', 'extract_data', 'logging', 'os', 'pd']


,doctor_id,doctor_name,specialty,ingested_at
0,100,Dr. Pérez,Vein,2025-11-02 17:49:00.187205+00:00
1,101,Dr. Gómez,Pain,2025-11-02 17:49:00.187205+00:00
2,102,Dr. Sánchez,Vein,2025-11-02 17:49:00.187205+00:00
3,103,Dr. Rodríguez,Pain,2025-11-02 17:49:00.187205+00:00
4,104,Dr. Martínez,Vein,2025-11-02 17:49:00.187205+00:00


,appointment_id,patient_id,doctor_id,appointment_date,status,ingested_at
0,1,10,100,2025-10-20,confirmed,2025-11-02 17:49:00.193328+00:00
1,2,11,101,2025-10-21,cancelled,2025-11-02 17:49:00.193328+00:00
2,3,12,100,2025-10-20,confirmed,2025-11-02 17:49:00.193328+00:00
3,4,13,101,2025-10-20,confirmed,2025-11-02 17:49:00.193328+00:00
4,5,14,102,2025-10-20,cancelled,2025-11-02 17:49:00.193328+00:00


In [434]:
# --- Fix foreign key issue ---
appointments_clean = appointments_clean[
    appointments_clean["doctor_id"].isin(doctors_clean["doctor_id"])
]

In [435]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS healthtech;")
connection.commit()

In [436]:
create_doctors_table = """
CREATE TABLE IF NOT EXISTS healthtech.doctors (
    doctor_id BIGINT PRIMARY KEY,
    doctor_name TEXT,
    specialty TEXT,
    ingested_at TIMESTAMPTZ
)
"""


In [437]:
    create_appointments_table = """
    CREATE TABLE IF NOT EXISTS healthtech.appointments (
        appointment_id BIGINT PRIMARY KEY,
        doctor_id BIGINT REFERENCES doctors(doctor_id),
        patient_id BIGINT,
        appointment_date DATE,
        status TEXT,
        ingested_at TIMESTAMPTZ
    )
    """

In [438]:
    cursor.execute(create_doctors_table)
    cursor.execute(create_appointments_table)
    connection.commit()
    print("Tables verified or created")

Tables verified or created


In [439]:
    upsert_doctors = """
        INSERT INTO healthtech.doctors (doctor_id, doctor_name, specialty, ingested_at)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (doctor_id)
        DO UPDATE SET
            doctor_name = EXCLUDED.doctor_name,
            specialty = EXCLUDED.specialty,
            ingested_at = EXCLUDED.ingested_at
    """

In [440]:
    upsert_appointments = """
        INSERT INTO healthtech.appointments (appointment_id, doctor_id, patient_id, appointment_date, status,ingested_at)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (appointment_id)
        DO UPDATE SET
            doctor_id = EXCLUDED.doctor_id,
            patient_id = EXCLUDED.patient_id,
            appointment_date = EXCLUDED.appointment_date,
            status = EXCLUDED.status,
            ingested_at = EXCLUDED.ingested_at
    """

In [441]:
# === 4️⃣ Prepare Data for Insert ===
doctor_records = doctors_clean[["doctor_id", "doctor_name", "specialty","ingested_at"]].values.tolist()
appointment_records = appointments_clean[["appointment_id", "doctor_id", "patient_id", "appointment_date", "status","ingested_at"]].values.tolist()

# === 5️⃣ Execute UPSERTs ===
print(f"Upserting {len(doctor_records)} doctors...")
execute_batch(cursor, upsert_doctors, doctor_records, page_size=100)

print(f"Upserting {len(appointment_records)} appointments...")
execute_batch(cursor, upsert_appointments, appointment_records, page_size=100)

# === 6️⃣ Commit and Close ===
connection.commit()
print(" Data successfully upserted into PostgreSQL!")


Upserting 5 doctors...
Upserting 850 appointments...
 Data successfully upserted into PostgreSQL!
